# 🌟 Enriquecimiento de Datos con IA

Este notebook muestra cómo podemos utilizar la IA generativa para enriquecer nuestros datos, generar nuevas características y obtener insights adicionales.

## ¿Qué aprenderás?
- 🔍 Enriquecer datasets existentes con descripciones generadas por IA
- 🏷️ Clasificar automáticamente textos y categorías
- 🧪 Generar datos sintéticos para pruebas o entrenamiento
- 💡 Extraer insights automáticamente a partir de datos

## 📚 Importamos las bibliotecas necesarias

In [ ]:
# Instalamos las bibliotecas necesarias (descomenta si es necesario)
# !pip install openai python-dotenv pandas seaborn matplotlib

In [ ]:
# Importamos las bibliotecas
import openai
import os
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from dotenv import load_dotenv
import time  # Para añadir pausas entre llamadas a la API

# Configuración para visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)
%matplotlib inline

## 🔑 Configuración de la API de OpenAI

In [ ]:
# Cargamos la API key desde el archivo .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Verificamos y configuramos la API
if api_key is None or api_key == "tu_api_key_aqui":
    print("⚠️ ERROR: API key no configurada correctamente.")
    print("Por favor, crea un archivo .env con tu OPENAI_API_KEY.")
else:
    print("✅ API key cargada correctamente.")
    # Configuramos la biblioteca con la API key
    openai.api_key = api_key

In [ ]:
# Función para hacer peticiones a la API de OpenAI
def consultar_openai(prompt, modelo="gpt-3.5-turbo", temperatura=0.7, max_tokens=150, formato_salida=None):
    """Realiza una consulta a la API de OpenAI
    
    Args:
        prompt (str): El texto de entrada para el modelo
        modelo (str): El modelo a utilizar
        temperatura (float): Controla la creatividad (0.0 a 1.0)
        max_tokens (int): Longitud máxima de la respuesta
        formato_salida (str): Si se requiere un formato específico (json, lista, etc.)
        
    Returns:
        str o dict: El texto generado o un objeto JSON si se solicitó
    """
    try:
        # Añadimos instrucciones de formato si es necesario
        if formato_salida == "json":
            prompt += "\n\nDevuelve la respuesta en formato JSON válido."
            response_format = {"type": "json_object"}
        else:
            response_format = None
            
        # Creamos la petición al API
        response = openai.chat.completions.create(
            model=modelo,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperatura,
            max_tokens=max_tokens,
            response_format=response_format
        )
        
        # Extraemos el texto generado
        respuesta = response.choices[0].message.content
        
        # Si se solicitó JSON, convertimos la respuesta a un diccionario
        if formato_salida == "json":
            try:
                return json.loads(respuesta)
            except json.JSONDecodeError:
                print("⚠️ Advertencia: No se pudo decodificar el JSON. Devolviendo texto plano.")
                return respuesta
        else:
            return respuesta
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

## 🐧 Cargamos el Dataset de Pingüinos

Vamos a utilizar el dataset de pingüinos de Seaborn, que contiene información sobre diferentes especies de pingüinos en la Antártida.

In [ ]:
# Cargamos el dataset de pingüinos
penguins = sns.load_dataset("penguins")

# Mostramos las primeras filas
penguins.head()

In [ ]:
# Exploramos información básica del dataset
print(f"Número de filas: {penguins.shape[0]}")
print(f"Número de columnas: {penguins.shape[1]}")
print(f"Especies de pingüinos: {penguins['species'].unique()}")
print(f"Islas: {penguins['island'].unique()}")

# Verificamos valores nulos
print("\nValores nulos por columna:")
print(penguins.isnull().sum())

In [ ]:
# Eliminamos filas con valores nulos para este ejemplo
penguins_clean = penguins.dropna().reset_index(drop=True)
print(f"Dataset limpio: {penguins_clean.shape[0]} filas")

## 🔍 Enriquecimiento 1: Generar Descripciones Detalladas

Vamos a usar la IA para generar descripciones detalladas para cada especie de pingüino.

In [ ]:
# Obtenemos las especies únicas
especies = penguins_clean['species'].unique()
print(f"Especies a describir: {especies}")

In [ ]:
# Función para generar descripción de una especie
def generar_descripcion_especie(especie):
    prompt = f"""
    Genera una descripción detallada del pingüino {especie} que incluya:
    1. Características físicas principales
    2. Hábitat y distribución geográfica
    3. Comportamiento y alimentación
    4. Estado de conservación
    
    Formato: párrafo conciso de 100-150 palabras.
    """
    
    # Llamamos a la API
    return consultar_openai(prompt, temperatura=0.7, max_tokens=200)

In [ ]:
# Generamos descripciones para cada especie
descripciones = {}

for especie in especies:
    print(f"Generando descripción para: {especie}...")
    descripciones[especie] = generar_descripcion_especie(especie)
    print(f"✅ Descripción generada")
    # Pausa para evitar límites de tasa en la API
    time.sleep(1)

# Mostramos las descripciones
for especie, descripcion in descripciones.items():
    print(f"\n🐧 {especie.upper()}:\n")
    print(descripcion)

## 🏷️ Enriquecimiento 2: Clasificación Automática

Vamos a usar la IA para clasificar a los pingüinos en categorías de tamaño basadas en sus medidas.

In [ ]:
# Seleccionamos una muestra para trabajar (10 pingüinos)
muestra_pingüinos = penguins_clean.sample(10, random_state=42).reset_index(drop=True)
muestra_pingüinos.head()

In [ ]:
# Función para clasificar el tamaño del pingüino basado en sus medidas
def clasificar_tamaño(fila):
    prompt = f"""
    Basado en las siguientes medidas de un pingüino {fila['species']}, clasifícalo en una categoría de tamaño (Pequeño, Mediano, Grande):
    - Longitud del pico: {fila['bill_length_mm']} mm
    - Profundidad del pico: {fila['bill_depth_mm']} mm
    - Longitud de la aleta: {fila['flipper_length_mm']} mm
    - Masa corporal: {fila['body_mass_g']} g
    
    Devuelve solo una palabra: Pequeño, Mediano o Grande.
    """
    
    # Llamamos a la API con baja temperatura para respuestas consistentes
    return consultar_openai(prompt, temperatura=0.1, max_tokens=10)

In [ ]:
# Clasificamos cada pingüino en la muestra
categorias_tamaño = []

for idx, fila in muestra_pingüinos.iterrows():
    print(f"Clasificando pingüino {idx+1}/10...")
    categoria = clasificar_tamaño(fila)
    categorias_tamaño.append(categoria.strip())
    # Pausa para evitar límites de tasa en la API
    time.sleep(1)

# Añadimos la categoría al dataframe
muestra_pingüinos['categoria_tamaño'] = categorias_tamaño

# Mostramos el resultado
muestra_pingüinos[['species', 'body_mass_g', 'flipper_length_mm', 'categoria_tamaño']]

In [ ]:
# Visualizamos la distribución de categorías
plt.figure(figsize=(10, 6))
sns.countplot(x='categoria_tamaño', hue='species', data=muestra_pingüinos, palette='viridis')
plt.title('Categorías de Tamaño por Especie', fontsize=15)
plt.xlabel('Categoría de Tamaño', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.tight_layout()
plt.show()

## 🧪 Enriquecimiento 3: Generación de Datos Sintéticos

Vamos a utilizar la IA para generar datos de pingüinos sintéticos basados en los patrones del dataset original.

In [ ]:
# Primero, obtenemos un resumen estadístico del dataset
resumen = penguins_clean.groupby('species').agg({
    'bill_length_mm': ['mean', 'min', 'max'],
    'bill_depth_mm': ['mean', 'min', 'max'],
    'flipper_length_mm': ['mean', 'min', 'max'],
    'body_mass_g': ['mean', 'min', 'max']
}).round(2)

# Mostramos el resumen
resumen

In [ ]:
# Función para generar pingüinos sintéticos
def generar_pingüinos_sinteticos(n=5):
    # Convertimos el resumen a formato de texto para el prompt
    resumen_texto = ""
    for especie in especies:
        resumen_texto += f"\n{especie}:\n"
        for medida in ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']:
            media = resumen.loc[especie, (medida, 'mean')]
            minimo = resumen.loc[especie, (medida, 'min')]
            maximo = resumen.loc[especie, (medida, 'max')]
            resumen_texto += f"- {medida}: media={media}, rango=[{minimo}-{maximo}]\n"
    
    prompt = f"""
    Genera {n} pingüinos sintéticos basados en las siguientes estadísticas de pingüinos reales.
    Asegúrate de que los valores sean realistas y dentro de los rangos típicos para cada especie.
    
    Estadísticas por especie:
    {resumen_texto}
    
    Para cada pingüino, genera:
    1. species: Adelie, Chinstrap, o Gentoo
    2. island: Biscoe, Dream, o Torgersen
    3. bill_length_mm: longitud del pico en mm
    4. bill_depth_mm: profundidad del pico en mm
    5. flipper_length_mm: longitud de la aleta en mm
    6. body_mass_g: masa corporal en gramos
    7. sex: male o female
    
    Devuelve un array de {n} objetos JSON, uno por cada pingüino generado.
    """
    
    # Llamamos a la API solicitando formato JSON
    resultado = consultar_openai(prompt, temperatura=0.8, max_tokens=500, formato_salida="json")
    
    # Si recibimos un diccionario, extraemos la lista de pingüinos
    if isinstance(resultado, dict) and 'pingüinos' in resultado:
        return resultado['pingüinos']
    elif isinstance(resultado, list):
        return resultado
    else:
        # Intentamos extraer manualmente el JSON si no se devolvió correctamente
        try:
            texto = str(resultado)
            # Buscamos el primer [ y el último ]
            inicio = texto.find('[')
            fin = texto.rfind(']') + 1
            if inicio >= 0 and fin > inicio:
                json_text = texto[inicio:fin]
                return json.loads(json_text)
            else:
                print("❌ No se pudo extraer JSON de la respuesta.")
                return []
        except Exception as e:
            print(f"❌ Error al procesar JSON: {e}")
            return []

In [ ]:
# Generamos 5 pingüinos sintéticos
print("Generando pingüinos sintéticos...")
pingüinos_sinteticos = generar_pingüinos_sinteticos(n=5)

# Convertimos a DataFrame
if pingüinos_sinteticos:
    df_sinteticos = pd.DataFrame(pingüinos_sinteticos)
    print("\n✅ Pingüinos sintéticos generados:")
    display(df_sinteticos)
else:
    print("❌ No se pudieron generar pingüinos sintéticos.")

## 💡 Enriquecimiento 4: Extracción de Insights

Vamos a pedirle a la IA que analice nuestros datos y extraiga insights interesantes.

In [ ]:
# Generamos un resumen de los datos para el prompt
correlaciones = penguins_clean.corr().round(2)
conteo_especies = penguins_clean['species'].value_counts()
conteo_islas = penguins_clean.groupby(['island', 'species']).size().unstack(fill_value=0)

# Calculamos medias por especie para cada medida
medias_por_especie = penguins_clean.groupby('species').mean().round(2)

# Resumen de datos en formato texto
resumen_datos = f"""
RESUMEN DEL DATASET DE PINGÜINOS:

1. Distribución de especies:
{conteo_especies.to_string()}

2. Distribución por isla y especie:
{conteo_islas.to_string()}

3. Medias por especie:
{medias_por_especie.to_string()}

4. Matriz de correlación:
{correlaciones.to_string()}
"""

In [ ]:
# Función para extraer insights
def extraer_insights(resumen_datos):
    prompt = f"""
    Analiza el siguiente resumen de un dataset de pingüinos y extrae 5 insights relevantes y útiles.
    Para cada insight, proporciona:
    1. Una descripción clara del hallazgo
    2. Una posible explicación biológica o ecológica
    3. Una sugerencia de visualización que podría ilustrar mejor este insight
    
    {resumen_datos}
    
    Formato de respuesta: Devuelve un objeto JSON con una lista de 5 insights, cada uno con las propiedades 'hallazgo', 'explicacion' y 'visualizacion_sugerida'.
    """
    
    # Llamamos a la API solicitando formato JSON
    return consultar_openai(prompt, temperatura=0.7, max_tokens=800, formato_salida="json")

In [ ]:
# Extraemos insights de los datos
print("Extrayendo insights de los datos...")
insights = extraer_insights(resumen_datos)

# Mostramos los insights
if insights and 'insights' in insights:
    for i, insight in enumerate(insights['insights'], 1):
        print(f"\n✨ INSIGHT {i}:")
        print(f"🔍 Hallazgo: {insight['hallazgo']}")
        print(f"🧠 Explicación: {insight['explicacion']}")
        print(f"📊 Visualización sugerida: {insight['visualizacion_sugerida']}")
else:
    print("❌ No se pudieron extraer insights.")

## 🎨 Implementemos una de las visualizaciones sugeridas

In [ ]:
# Ejemplo de visualización basada en uno de los insights
plt.figure(figsize=(12, 8))

# Gráfico de dispersión con todas las medidas
sns.scatterplot(x='flipper_length_mm', y='body_mass_g', 
                hue='species', size='bill_length_mm',
                sizes=(20, 200), alpha=0.8, palette='viridis',
                data=penguins_clean)

plt.title('Relación entre Longitud de Aleta, Masa Corporal y Longitud del Pico por Especie', fontsize=16)
plt.xlabel('Longitud de Aleta (mm)', fontsize=14)
plt.ylabel('Masa Corporal (g)', fontsize=14)
plt.legend(title='Especie', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 🌱 Cómo la IA apoya en enriquecer flujos de datos

La IA puede enriquecer tus flujos de datos de múltiples formas:

### 1️⃣ Clasificación de texto
- 🏷️ Categorización automática de comentarios, reseñas o documentos
- 📊 Análisis de sentimiento para entender la opinión de clientes
- 🔍 Extracción de temas y conceptos clave de grandes volúmenes de texto

### 2️⃣ Generación de datos
- 🧪 Creación de datos sintéticos para pruebas o entrenamiento
- 📝 Generación de descripciones, títulos o resúmenes
- 🔄 Aumento de datos para equilibrar clases subrepresentadas

### 3️⃣ Limpieza de datos
- 🧹 Detección y corrección de errores o valores atípicos
- 🔄 Normalización de texto (nombres, direcciones, etc.)
- 📋 Completar valores faltantes con datos realistas

### 4️⃣ Extracción de insights
- 💡 Descubrimiento automático de patrones y relaciones
- 📈 Generación de hipótesis para investigación adicional
- 📊 Sugerencias de visualizaciones relevantes

## 🚀 Ideas "wow" para demostraciones

### 1️⃣ Clasificación de reseñas en vivo
- Crear un pequeño dashboard donde se puedan pegar reseñas de productos
- La IA analiza en tiempo real y clasifica por sentimiento, temas principales y problemas detectados
- Muestra un resumen visual con gráficos de temas y sentimientos

### 2️⃣ Generación de insights a partir de descripciones cortas
- Permitir al usuario describir brevemente un negocio o producto
- La IA genera automáticamente:
  - Posibles segmentos de clientes
  - Indicadores clave de rendimiento (KPIs) relevantes
  - Ideas para análisis de datos y experimentos

### 3️⃣ Transformación de datos numéricos en narrativas
- Cargar un dataset simple (ventas, usuarios, etc.)
- La IA genera automáticamente una narrativa que explica los patrones principales
- Incluye recomendaciones accionables basadas en los datos

### 4️⃣ Generador de dashboards por voz
- El usuario describe verbalmente qué quiere visualizar
- La IA genera el código para crear ese dashboard específico
- Se muestra en tiempo real el resultado visual

## 📝 Ejemplo de integración con dashboard

Aquí un ejemplo de cómo podrías integrar el análisis de pingüinos en un dashboard:

In [ ]:
# Ejemplo conceptual de código para Streamlit (no ejecutar)
'''
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import openai
import os
from dotenv import load_dotenv

# Configuración inicial
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
st.set_page_config(page_title="Análisis de Pingüinos con IA", layout="wide")

# Título y descripción
st.title("🐧 Análisis de Pingüinos Enriquecido con IA")
st.write("Explora datos de pingüinos con ayuda de inteligencia artificial generativa")

# Cargamos datos
penguins = sns.load_dataset("penguins")
penguins_clean = penguins.dropna().reset_index(drop=True)

# Sidebar para filtros
st.sidebar.header("Filtros")
selected_species = st.sidebar.multiselect("Especies", penguins_clean['species'].unique(), default=penguins_clean['species'].unique())
selected_island = st.sidebar.multiselect("Islas", penguins_clean['island'].unique(), default=penguins_clean['island'].unique())

# Filtramos datos
filtered_data = penguins_clean[
    penguins_clean['species'].isin(selected_species) &
    penguins_clean['island'].isin(selected_island)
]

# Columnas principales
col1, col2 = st.columns([3, 2])

with col1:
    st.header("Visualización de Datos")
    
    # Selector de visualización
    chart_type = st.selectbox(
        "Selecciona tipo de gráfico", 
        ["Scatter Plot", "Box Plot", "Violin Plot", "Pair Plot"]
    )
    
    # Generamos el gráfico seleccionado
    fig, ax = plt.subplots(figsize=(10, 6))
    
    if chart_type == "Scatter Plot":
        sns.scatterplot(x='flipper_length_mm', y='body_mass_g', 
                        hue='species', size='bill_length_mm',
                        sizes=(20, 200), alpha=0.8, data=filtered_data, ax=ax)
        plt.title('Relación entre Longitud de Aleta y Masa Corporal')
        
    elif chart_type == "Box Plot":
        sns.boxplot(x='species', y='body_mass_g', data=filtered_data, ax=ax)
        plt.title('Distribución de Masa Corporal por Especie')
        
    elif chart_type == "Violin Plot":
        sns.violinplot(x='species', y='flipper_length_mm', data=filtered_data, ax=ax)
        plt.title('Distribución de Longitud de Aleta por Especie')
        
    elif chart_type == "Pair Plot":
        # Para pairplot necesitamos una figura diferente
        plt.close(fig)
        fig = sns.pairplot(filtered_data, hue='species', height=2.5)
        plt.suptitle('Relaciones entre Variables por Especie', y=1.02)
    
    st.pyplot(fig)

with col2:
    st.header("Análisis con IA")
    
    # Si el usuario selecciona una sola especie, mostramos su descripción
    if len(selected_species) == 1:
        st.subheader(f"Sobre los pingüinos {selected_species[0]}")
        
        # Aquí utilizaríamos la función de generar_descripcion_especie
        # y guardaríamos en caché el resultado para no llamar a la API cada vez
        if "descripciones" not in st.session_state:
            st.session_state.descripciones = {}
            
        if selected_species[0] not in st.session_state.descripciones:
            with st.spinner("Generando descripción con IA..."):
                # Llamaríamos a la función que usa OpenAI para generar la descripción
                st.session_state.descripciones[selected_species[0]] = "Descripción del pingüino..." # Simulado
                
        st.write(st.session_state.descripciones[selected_species[0]])
    
    # Botón para generar insights sobre los datos filtrados
    if st.button("Generar insights con IA"):
        with st.spinner("El modelo está analizando los datos..."):
            # Aquí llamaríamos a la función que extrae insights
            # Simulado para este ejemplo
            insights = ["El pingüino Gentoo es significativamente más pesado que las otras especies",
                       "Existe una fuerte correlación entre la longitud de la aleta y la masa corporal",
                       "La distribución de especies varía considerablemente según la isla"]
            
            for insight in insights:
                st.info(insight)
    
    # Cuadro de texto para preguntas del usuario
    st.subheader("Pregunta sobre los datos")
    user_question = st.text_input("Haz una pregunta sobre los pingüinos:")
    
    if user_question:
        with st.spinner("Procesando tu pregunta..."):
            # Aquí llamaríamos a la API de OpenAI con el contexto de los datos
            # Simulado para este ejemplo
            response = "Los pingüinos Gentoo tienen las aletas más largas de las tres especies, con un promedio de 217mm comparado con 190mm para Adelie y 195mm para Chinstrap."
            st.success(response)
'''

## 🌟 Conclusión

En este notebook hemos explorado diferentes formas de enriquecer datos con IA generativa:

- ✅ Generamos descripciones detalladas para complementar datos numéricos
- ✅ Clasificamos automáticamente ejemplos según sus características
- ✅ Creamos datos sintéticos basados en patrones existentes
- ✅ Extrajimos insights automáticos a partir de estadísticas

Estas técnicas pueden aplicarse a cualquier dominio y tipo de datos, permitiéndote:

- 🔍 Descubrir patrones ocultos en tus datos
- 🚀 Acelerar análisis exploratorios
- 💡 Generar hipótesis para investigación adicional
- 📊 Crear visualizaciones y dashboards más informativos

### Próximos pasos:
- Prueba estas técnicas con tus propios datasets
- Integra estos enriquecimientos en tus flujos de análisis de datos
- Crea dashboards interactivos que combinen datos y generación de IA